In [ ]:
# AiiDA imports.
%load_ext aiida
%aiida
from aiida import common, orm
from surfaces_tools.widgets import series_plotter

In [1]:
from IPython.display import display, clear_output
import utils
import ipywidgets as ipw
from app_widgets import AppWidgets

In [62]:
VIEWERS = {
    "CP2K_AdsorptionE": "view_adsorption_energy.ipynb",
    "CP2K_GeoOpt": "view_geometry_optimization.ipynb",
    "CP2K_CellOpt": "view_geometry_optimization.ipynb",
    "CP2K_ORBITALS": "view_orbitals.ipynb",
    "CP2K_PDOS": "view_pdos.ipynb",
    "CP2K_STM": "view_stm.ipynb",
    "CP2K_AFM": "view_afm.ipynb",
    "CP2K_HRSTM": "view_hrstm.ipynb",
    "CP2K_Phonons": "view_phonons.ipynb",
    "CP2K_NEB": "view_neb.ipynb",
    "CP2K_Replica": "view_replica.ipynb",
    "ReplicaWorkChain": "view_replica.ipynb",
}

def uuids_to_nodesdict(uuids):
    workflows = {}
    nworkflows = 0
    for uuid in uuids:
        try:
            node = orm.load_node(uuid)
            nodeisobsolete = "obsolete" in node.extras and node.extras["obsolete"]
            if node.label in VIEWERS and not nodeisobsolete:
                nworkflows += 1
                if node.label in workflows:
                    workflows[node.label].append(node)
                else:
                    workflows[node.label] = [node]
        except common.NotExistent:
            pass

    return nworkflows, workflows

def get_all_structures_and_geoopts(node):
    """Get all atomistic models that led to the one used in the STM simulation"""
    current_node = node
    all_structures = [node]
    all_geoopts = []
    while current_node is not None:
        if isinstance(current_node, orm.StructureData):
            current_node = current_node.creator
        elif isinstance(current_node, orm.CalcJobNode):
            current_node = current_node.caller
            
        elif isinstance(current_node, orm.WorkChainNode):
            if "GeoOpt" in current_node.label:
                all_geoopts.append(current_node)
                current_node = current_node.inputs.structure
                all_structures.append(current_node)
            else:
                current_node = current_node.caller
    
    return all_structures, all_geoopts

def get_workflows(start_date, end_date):
    qb = orm.QueryBuilder()
    qb.append(
        orm.StructureData,
        filters={
            "extras": {"has_key": "surfaces"},
            "mtime": {"and": [{"<=": end_date}, {">": start_date}]},
        },
    )
    qb.order_by({orm.StructureData: {"mtime": "desc"}})

    # For each structure in QB create a dictionary with info on the workflows computed on it.
    data = []
    for node in qb.all(flat=True):
        # print("node ", node.pk, " extras ", node.extras["surfaces"])
        extras = node.extras["surfaces"]
        nworkflows = 0
        if isinstance(extras, list):
            nworkflows, workflows = uuids_to_nodesdict(node.extras["surfaces"])
        if nworkflows > 0:
            data.append(workflows)
    
    return data

In [2]:
# Widgets
config = utils.read_json("config.json")

# Send simulation to openBIS

In [3]:
# Initialise app widgets
app_widgets = AppWidgets("config.json")
app_widgets.load_dropdown_lists()

## Select experiment and molecule

## Select simulation

In [ ]:
import datetime as datetime
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=20)
data = get_workflows(start_date, end_date)

all_workflows = []
for workflows in data:
    workflows = workflows.values()
    workflows = list(workflows)[0]
    all_workflows.extend(workflows)

workflows_dropdown = utils.Dropdown(description='Workflows', disabled=False, layout = ipw.Layout(width = '993px'), style = {'description_width': "110px"}, options = all_workflows)
display(workflows_dropdown)

In [ ]:
def upload_simulations(b):
    # Get STM Simulation Workchain from AiiDA
    stm_workchain_node = workflows_dropdown.value
    structure_stm = stm_workchain_node.inputs.structure
    structure_stm_pk = structure_stm.pk
    structure_stm_node = load_node(structure_stm_pk)

    # Get Geometry Optimisation Workchain from AiiDA
    all_atom_mods, all_geoopts = get_all_structures_and_geoopts(structure_stm_node)
    all_exist_openbis_atom_mods, all_exist_openbis_geoopts, all_exist_openbis_stms = [], [], []
    
    # Get all simulations and atomistic models from openbis
    openbis_geoopts = app_widgets.session.get_objects(type = "GEOMETRY_OPTIMISATION")
    openbis_atom_mods = app_widgets.session.get_objects(type = "ATOMISTIC_MODEL")
    
    # Verify which GeoOpts are already in openBIS
    for aiida_geoopt_idx, aiida_geoopt in enumerate(all_geoopts):
        exists_openbis_geoopt = False
        for openbis_geoopt in openbis_geoopts:
            if openbis_geoopt.props.get("wfms-uuid") == aiida_geoopt.uuid:
                all_exist_openbis_geoopts.append([openbis_geoopt, True])
                exists_openbis_geoopt = True
                print(f"GeoOpt {openbis_geoopt.props.get('wfms-uuid')} already exists.")

        if exists_openbis_geoopt == False:
            all_exist_openbis_geoopts.append([None, False])
    
    # Reverse the lists because in openBIS, one should start by building the parents.
    all_geoopts.reverse()
    all_exist_openbis_geoopts.reverse()
    
    # Verify which structures (atomistic models) are already in openBIS
    for aiida_atom_mod_idx, aiida_atom_mod in enumerate(all_atom_mods):
        exists_openbis_atom_mod = False
        for openbis_atom_mod in openbis_atom_mods:
            if openbis_atom_mod.props.get("wfms-uuid") == aiida_atom_mod.uuid:
                all_exist_openbis_atom_mods.append([openbis_atom_mod, True])
                exists_openbis_atom_mod = True
                print(f"Atomistic model {openbis_atom_mod.props.get('wfms-uuid')} already exists.")
            
        if exists_openbis_atom_mod == False:
            all_exist_openbis_atom_mods.append([None, False])
    
    # Reverse the lists because in openBIS, one should start by building the parents.
    all_atom_mods.reverse()
    all_exist_openbis_atom_mods.reverse()
    
    # Verify which simulations are already in openBIS
    stm_simulation_exists = False
    for simulation in simulations_openbis:
        if simulation.props.get("wfms-uuid") == stm_workchain_node.uuid:
            all_simulations_exist_openbis.append([simulation, True])
            print(f"STM simulation {simulation.props.get('wfms-uuid')} already exists.")
        
    if stm_simulation_exists == False:
        all_simulations_exist_openbis.append([None, False])
    
    # Build atomistic models (structures in AiiDA) in openBIS
    all_atomistic_models = []
    
    for structure_index, structure in enumerate(all_structures):
        structure_exist_openbis = all_structures_exist_openbis[structure_index]
        
        if structure_exist_openbis[1] == False:
            # Get Structure details from AiiDA
            structure_ase = structure.get_ase()
            # structure_ase.positions # Atoms positions
            # structure_ase.symbols # Atoms Symbols
            # structure_ase.pbc # PBC (X, Y, Z)
            # structure.cell # Cell vectors
            
            # Create Atomistic Model in openBIS
            atomistic_model = session.new_sample(collection = atomistic_models_collection, type='ATOMISTIC_MODEL')
            
            atomistic_model.props['$name'] = "Atomistic Model 1"
            atomistic_model.props['description'] = "A nice atomistic model"
            atomistic_model.props['wfms-uuid'] = structure.uuid
            atomistic_model.props['pbc-x'] = int(structure_ase.pbc[0])
            atomistic_model.props['pbc-y'] = int(structure_ase.pbc[1])
            atomistic_model.props['pbc-z'] = int(structure_ase.pbc[2])
            
            if len(all_structures) > 1 and structure_index == len(all_structures) - 1: # If it is the last of more than one structures, it is optimised.
                atomistic_model.props['optimised'] = 1
            else:
                atomistic_model.props['optimised'] = 0
            
            atomistic_model.save()
            
            all_atomistic_models.append(atomistic_model)
        else:
            all_atomistic_models.append(structure_exist_openbis[0])
    
    # Build GeoOpts in openBIS
    all_geoopts_models = []
    
    for geoopt_index, geoopt in enumerate(all_geoopts):
        geoopt_exist_openbis = all_geoopts_exist_openbis[geoopt_index]
        
        if geoopt_exist_openbis[1] == False:
            parent_atomistic_model = all_atomistic_models[geoopt_index]
            geoopt_simulation = session.new_sample(experiment = selected_experiment_identifier, type = 'SIMULATION', parents = [parent_atomistic_model])
            geoopt_simulation.props['$name'] = "GeoOpt Simulation"
            geoopt_simulation.props['description'] = "GeoOpt Simulation"
            geoopt_simulation.props['wfms-uuid'] = geoopt.uuid
            geoopt_simulation.save()
            
            geoopt_model = session.new_sample(experiment = selected_experiment_identifier, type = 'GEO_OPTIMISATION', parents = [geoopt_simulation])
            
            geoopt_model.save()
            
            # Its plus one because there are N+1 geometries for N GeoOpts
            all_atomistic_models[geoopt_index + 1].add_parents(geoopt_model)
            all_atomistic_models[geoopt_index + 1].save()
            
            all_geoopts_models.append(geoopt_model)
        else:
            all_geoopts_models.append(geoopt_exist_openbis[0])
    
    # Build STM simulation in openBIS
    
    if all_simulations_exist_openbis[0][1] == False:
        # Simulated Scanning Tunneling Microscopy
        simulation_model = session.new_sample(experiment = selected_experiment_identifier, type = 'SIMULATION')
        simulation_model.props['$name'] = "Simulation STM"
        simulation_model.props['description'] = "Simulation"
        simulation_model.props['wfms-uuid'] = stm_workchain_node.uuid
        simulation_model.save()

        optimised_atomistic_model = all_atomistic_models[-1]
        stm_simulation_model = session.new_sample(experiment = selected_experiment_identifier, type = 'STM', parents = [simulation_model, optimised_atomistic_model])
        stm_simulation_uuid = stm_workchain_node.uuid
        dft_params = dict(stm_workchain_node.inputs.dft_params)
        
        #TODO: Remove this is the future. Orbitals do not have stm_params
        try:
            stm_params = dict(stm_workchain_node.inputs.stm_params)
            stm_simulation_model.props['emin-J'] = stm_params['--energy_range'][0] * 1.60217663e-19
            stm_simulation_model.props['emax-J'] = stm_params['--energy_range'][1] * 1.60217663e-19
            stm_simulation_model.props['de-J'] = stm_params['--energy_range'][2] * 1.60217663e-19
        except:
            pass
        
        stm_simulation_model.props['$name'] = "Simulated STM"
        stm_simulation_model.props['description'] = "A nice simulated STM"
        stm_simulation_model.save()
        
        stm_simulation_images_zip_filename = series_plotter_inst.create_zip_link_for_openbis()
        
        stm_simulation_images_dataset = session.new_dataset(
            type = "RAW_DATA", 
            files = [stm_simulation_images_zip_filename],
            sample = stm_simulation_model
        )
        stm_simulation_images_dataset.save()

        #TODO: How to do this using Python commands?
        stm_simulation_dataset_filename = "stm_simulation.aiida"
        os.system(f"verdi archive create {stm_simulation_dataset_filename} --no-call-calc-backward --no-call-work-backward --no-create-backward -N {stm_workchain_pk}")

        stm_simulation_dataset = session.new_dataset(
            type = "RAW_DATA", 
            files = [stm_simulation_dataset_filename],
            sample = stm_simulation_model
        )
        stm_simulation_dataset.save()

        # Delete the file after uploading
        os.remove(stm_simulation_dataset_filename)

In [ ]:
# display(app_widgets.object_dropdown)
display(app_widgets.save_close_buttons_hbox)
app_widgets.create_button.on_click(upload_simulations)
app_widgets.quit_button.on_click(app_widgets.close_notebook)